<a href="https://colab.research.google.com/github/babymlin/Sentiment/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

     |████████████████████████████████| 686kB 7.7MB/s 
84131840/84125825 [==============================] - 1s 0us/step


In [ ]:
import os
import pandas as pd
from glob import glob
def get_data(base="train"):
  dn = os.path.split(dataset)[0]
  fn = os.path.join(dn , "aclImdb", base, "pos", "*.txt")
  fns = glob(fn)
  contents, target = [], []
  for f in fns:
    with open(f, "r", encoding="utf-8") as file:
      contents.append(file.read())
      target.append(1)
  fn = os.path.join(dn , "aclImdb", base, "neg", "*.txt")
  fns = glob(fn)
  for f in fns:
    with open(f, "r", encoding="utf-8") as file:
      contents.append(file.read())
      target.append(0)
  df = pd.DataFrame({
      "contents":contents,
      "ans":target
  })
  return df["contents"], df["ans"]
x_train, y_train = get_data(base="train")
x_test, y_test = get_data(base="test")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(x_train)
x_train_seq = tok.texts_to_sequences(x_train)
x_test_seq = tok.texts_to_sequences(x_test)
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,10,13,576,9,13,28,4,145,342.0,22.0,116.0,9.0,39.0,22.0,780.0,9.0,99.0,70.0,10.0,444.0,9.0,574.0,996.0,1957.0,2.0,772.0,2.0,29.0,35.0,10.0,239.0,25.0,54.0,1293.0,30.0,29.0,51.0,9.0,263.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,6,3,62,4,104,2512,2,3.0,1128.0,264.0,15.0,65.0,93.0,142.0,341.0,151.0,2.0,1559.0,2043.0,2682.0,182.0,295.0,577.0,2.0,1128.0,36.0,1.0,2.0,137.0,80.0,1.0,5.0,27.0,16.0,65.0,220.0,2718.0,1.0,1045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109,12,11,17,44,3,173,4,475.0,718.0,2425.0,10.0,101.0,9.0,141.0,46.0,50.0,1105.0,71.0,9.0,44.0,74.0,345.0,20.0,894.0,7.0,7.0,42.0,126.0,1117.0,221.0,9.0,755.0,705.0,16.0,380.0,2.0,585.0,18.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,6,32,318,19,15,665,642,665.0,202.0,448.0,10.0,101.0,12.0,1862.0,119.0,3.0,84.0,289.0,8.0,11.0,19.0,198.0,238.0,339.0,467.0,2.0,2078.0,5.0,27.0,3.0,84.0,56.0,44.0,3.0,52.0,1264.0,2.0,3.0,77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2444,70,90,659,20,10,25,207,421.0,11.0,1479.0,5.0,19.0,228.0,1005.0,41.0,3.0,50.0,1311.0,2.0,396.0,31.0,2234.0,1.0,1830.0,177.0,80.0,139.0,1077.0,2.0,488.0,5.0,341.0,7.0,7.0,10.0,241.0,249.0,88.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,11,44,32,218,640,917,922,111,18.0,42.0,1050.0,16.0,336.0,455.0,2.0,315.0,299.0,6.0,2270.0,464.0,1.0,580.0,4.0,3.0,864.0,62.0,2.0,455.0,16.0,38.0,673.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,2761,16,1,2538,6,3,725,47,705.0,12.0,112.0,183.0,5.0,166.0,98.0,2868.0,54.0,548.0,86.0,2731.0,174.0,2.0,1049.0,350.0,10.0,188.0,262.0,3.0,760.0,4.0,58.0,365.0,69.0,80.0,316.0,11.0,30.0,1.0,99.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,832,111,2,104,2041,226,90,102,165.0,37.0,602.0,2961.0,5.0,132.0,11.0,90.0,9.0,875.0,5.0,231.0,15.0,98.0,4.0,1.0,102.0,258.0,1.0,26.0,605.0,2.0,914.0,50.0,37.0,3.0,1069.0,8.0,2733.0,10.0,479.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,1,83,317,4,11,307,13,1,115.0,204.0,107.0,2.0,10.0,101.0,204.0,107.0,172.0,307.0,4.0,1012.0,123.0,90.0,1.0,940.0,4.0,1546.0,2.0,106.0,68.0,92.0,9.0,13.0,14.0,148.0,291.0,298.0,446.0,11.0,6.0,617.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
seq_len = 512
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=seq_len)
x_test_pad = pad_sequences(x_test_seq, maxlen=seq_len)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
  #input_length輸入每一個詞彙時，有input_dim=3001個字數，降成128個特徵維度，參數總量=3001*128=384128
  Embedding(input_dim=3001, output_dim=128, mask_zero=True, input_length=seq_len),
  GlobalAveragePooling1D(),
  Dense(units=2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
Radam = tfa.optimizers.RectifiedAdam()
Ranger = tfa.optimizers.Lookahead(Radam)
model.compile(
    optimizer=Ranger,
    loss="sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
callbacks = [
  CSVLogger("logger.csv"),
  EarlyStopping(patience=5, monitor="val_loss", restore_best_weights=True),
  ModelCheckpoint("best-model.h5", save_best_only=True)
]

In [ ]:
batch_size=256
epochs = 100
model.fit(
  x_train_pad, y_train,
  batch_size=batch_size,
  epochs=epochs,
  callbacks=callbacks,
  validation_split=0.1
)

Epoch 1/100
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
88/88 [==============================] - 8s 44ms/step - loss: 0.6914 - accuracy: 0.5326 - val_loss: 0.7170 - val_accuracy: 8.0000e-04
Epoch 2/100
88/88 [==============================] - 4s 42ms/step - loss: 0.6827 - accuracy: 0.5556 - val_loss: 0.7559 - val_accuracy: 0.0016
Epoch 3/100
88/88 [==============================] - 4s 42ms/step - loss: 0.6688 - accuracy: 0.5582 - val_loss: 0.7755 - val_accuracy: 0.0196
Epoch 4/100
88/88 [==============================] - 4s 43ms/step - loss: 0.6469 - accuracy: 0.6011 - val_loss: 0.7596 - val_accuracy: 0.2104
Epoch 5/100
88/88 [==============================] - 4s 42ms/step - loss: 0.6149 - accuracy: 0.7003 - val_loss: 0.7013 - val_accuracy: 0.4908
Epoch 6/100
88/88 [==============================] - 4s 42ms/step - loss: 0.5755 - accuracy: 0.7706 - val_loss: 0.6799 - val_accuracy: 0.5664
Epoch 

In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.2911 - accuracy: 0.8805


[0.2910565733909607, 0.8805199861526489]

In [ ]:
layers = [
    # 每個詞彙都有128個係數, 3001 * 128 = 384128
    # !!!tokenize有多少個可調(上面要記得調): 3000 -> 4000
    # !!!詞向量維度可調(簡單小一點, 困難大一點): 128 -> 256/64
    # !!!input_length(上面要記得調): 512 -> 256
    Embedding(3001, 128, mask_zero=True)
]
w2v = Sequential(layers)
w2v.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
w = model.layers[0].get_weights()
w2v.layers[0].set_weights(w)

In [ ]:
from scipy.spatial.distance import cosine
from numpy.linalg import norm
import numpy as np

#1 word 1
tok1 = tok.word_index["scary"]
word1 = [[tok1]]
v1 = w2v.predict(word1)[0][0]
#2 word 2
tok2 = tok.word_index["horror"]
word2 = [[tok2]]
v2 = w2v.predict(word1)[0][0]
#cosine
print(1 - cosine(v1, v2))
print(np.dot(v1, v2) / (norm(v1) * norm(v2)))

1.0
1.0000001
